In [24]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [25]:
url = 'https://phoenix.overdrive.com/collection/25972'

In [26]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)

In [32]:
container = driver.find_element(By.XPATH, ".//section[contains(@id, 'search')]")
all_books = container.find_elements(By.XPATH, ".//li[contains(@class, 'js-titleCard Item')]")
#all_books[0].find_element(By.XPATH, ".//h3[contains(@class, 'title-name')]").text  # book title
#all_books[0].find_element(By.XPATH, ".//p[contains(@class, 'title-author')]").text.replace('by ', '')  # author
#all_books[0].find_element(By.XPATH, ".//div[contains(@class, 'title-header-bar')]").text  # availability
#all_books[0].find_element(By.XPATH, ".//span[contains(@class, 'title-format-badge')]").text  # book format
#all_books[0].find_element(By.XPATH, ".//div[contains(@class, 'TitleInfo')]/h3/a").get_attribute('href')  # book link

pagination = driver.find_element(By.XPATH, './/ul[contains(@class, "Pagination-itemsContainer")]')
pagination.find_elements(By.XPATH, './/a[contains(@class, "Pagination-item")]').txt #[-1].text


AttributeError: 'list' object has no attribute 'text'

In [28]:
book_titles = []
book_authors = []
book_availability = []
book_formats = []
book_links = []

current_page = 1
driver.get(url)
pagination = driver.find_element(By.XPATH, './/ul[contains(@class, "Pagination-itemsContainer")]')
last_page = pagination.find_elements(By.XPATH, './/a[contains(@class, "Pagination-item")]')[-1].text

while current_page <= int(last_page):
    container = driver.find_element(By.XPATH, ".//section[contains(@id, 'search')]")
    all_books = container.find_elements(By.XPATH, ".//li[contains(@class, 'js-titleCard Item')]")

    for book in all_books:
        title = book.find_element(By.XPATH, ".//h3[contains(@class, 'title-name')]").text
        book_titles.append(title)

        author = book.find_element(By.XPATH, ".//p[contains(@class, 'title-author')]").text.replace('by ', '')
        book_authors.append(author)

        availability = book.find_element(By.XPATH, ".//div[contains(@class, 'title-header-bar')]").text
        book_availability.append(availability)

        book_format = book.find_element(By.XPATH, ".//span[contains(@class, 'title-format-badge')]").text
        book_formats.append(book_format)

        link = book.find_element(By.XPATH, ".//div[contains(@class, 'TitleInfo')]/h3/a").get_attribute('href')
        book_links.append(link)

    try:
        next_button = driver.find_element(By.XPATH, './/a[contains(@class, "Pagination-item") and @aria-label="Next page"]')
        next_button.click()
        current_page += 1

    except:
        pass
 
driver.quit()


ValueError: invalid literal for int() with base 10: ''